## TAREA

* Con el código utilizado en clase, implementar una clase de Python similar a esta. Se le deben pasar como argumentos:
  - Cantidad de filtros
  - Tamaño de los filtros
  - Stride
  - Tipo de padding (“valid” o “same”)
* Debe tener un método “forward”, al cual se le pasa la matriz de entrada y devuelve la salida.
*Y un método “backward”, al cual se le pasa una matriz con $∂L/∂O$ y devuelve dos matrices, una con $∂L/∂F$ y otra con $∂L/∂X$.

In [ ]:
import numpy as np
import torch

In [ ]:
# Función para padding con ceros

def zero_padding(X, pad):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (batch_size, n_C, n_H, n_W)
    pad: Entero representando la cantidad de filas y columnas que se deben agregar con ceros

  Retorna:
    X_padded: Array numpy con dimensiones (batch_size, n_C, n_H + 2*pad, n_W + 2*pad)
  """

  X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))

  return X_padded

In [ ]:
# Función para realizar la operación de convolución

def convolve(X, W, b):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
    W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
    b: Entero con el valor de bias de la capa actual

  Retorna:
    Z: Entero con el valor del resultado
  """

  # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
  aux = X * W
  # Realizo la suma de todos los elementos
  aux = np.sum(aux)
  # Le sumo el valor del bias para obtener Z
  Z = aux + float(b)

  return Z

In [ ]:
class conv2d():
  def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
    """
    Argumentos:
      in_channels: n_C_prev
      out_channels: n_C
      kernel_size: tamaño del filtro
      stride: Entero con el valor de stride utilizado en la capa actual.
      padding: "same" o "valid"
    """

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding
    
    #  W: Array numpy con los pesos de los filtros utilizados en la capa actual (n_C, n_C_prev, filter_size, filter_size)
    self.W = np.random.randn(out_channels, in_channels, kernel_size, kernel_size)
    # b: Array numpy con los valores de bias utilizados en la capa actual (1, 1, 1, n_C)
    self.b = np.random.randn(1, 1, 1, out_channels)

    

  def forward(self, layer_input):
    """
    Argumentos:
      layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev)
    Retorna:
      Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
    """

    # Obtengo las dimensiones de la entrada
    (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

    # Obtengo las dimensiones de los filtros
    (n_C, n_C_prev, filter_size, filter_size) = self.W.shape

    # Calculo las dimensiones del volumen de salida de la capa actual

    if self.padding == "same":
      p = (self.kernel_size - 1)/2 # padding necesario para no alterar el tamaño de salida.
      n_H = int((n_H_prev + 2*p - self.kernel_size)/self.stride + 1)
      n_W = int((n_W_prev + 2*p - self.kernel_size)/self.stride + 1)

    elif self.padding == "valid":
      p = 0
      n_H = int((n_H_prev + 2*p - self.kernel_size)/self.stride + 1)
      n_W = int((n_W_prev + 2*p - self.kernel_size)/self.stride + 1)
    

    # Inicializo el volumen de salida con ceros
    Z = np.zeros([batch_size, n_C, n_H, n_W])

    # Agrego padding con ceros al volumen de entrada
    layer_input_padded = zero_padding(layer_input, p)

    # Comienzo iterando sobre cada ejemplo del batch
    for i in range(batch_size):

      # Itero sobre el eje vertical del volumen de salida
      for h in range(n_H):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = self.stride * h
        y_end = y_start + filter_size

        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W):
          # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          x_start = self.stride * w
          x_end = x_start + filter_size

          # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
          slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
          
          # Itero sobre la cantidad de canales del volumen de salida
          for c in range(n_C):

            # Obtengo el valor del filtro y bias del canal correspondiente
            filter = self.W[c, :, :, :]
            bias = self.b[c]

            # Computo la operación de convolución para esta ventana
            Z[i, c, h, w] = convolve(slice_from_input_padded, filter, bias)
    
    return Z


    def backward(self, grad_loss_output):
      """
      Argumentos:
        grad_loss_output: ∂L/∂O (1, n_C, n_H, n_W)
      Retorna:
        grad_loss_filter, grad_loss_input: (n_C, n_C_prev, filter_size, filter_size), (1, n_C_prev, n_H_prev, n_W_prev)
      """

      #grad_loss_input
      
      (1, n_C, n_H, n_W) = grad_loss_output.shape
      # dilatacion:
      aux_loss_out = np.zeros([1, n_C, n_H + self.stride-1, n_W + self.stride-1])
      aux_loss_out_padded = np.pad(aux_loss_out, ((0,0), (0,0), (kernel_size-1, kernel_size-1), (kernel_size-1, kernel_size-1)), mode='constant', constant_values = (0,0))
      
      w_flipped = np.flip(np.flip(self.W, 2),3)

      (n_C, n_C_prev, filter_size, filter_size) = w_flipped.shape
      (1, n_C, n_H_padded, n_W_padded) = aux_loss_out_padded.shape
    

      n_H_prev = int((n_H_padded - filter_size) + 1)
      n_W_prev = int((n_W_padded - filter_size) + 1)
      grad_loss_input = np.zeros([1, n_C, n_H_prev, n_W_prev])

      # convolve
      
      # Itero sobre el eje vertical del volumen de salida
      for h in range(n_H_padded):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = h
        y_end = y_start + filter_size

        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W_padded):
          # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          x_start =  w
          x_end = x_start + filter_size

          # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
          slice_from_input_padded = aux_loss_out_padded[i, :, y_start:y_end, x_start:x_end]
          
          # Itero sobre la cantidad de canales del volumen de salida
          for c in range(n_C):

            # Obtengo el valor del filtro y bias del canal correspondiente
            filter = w_flipped[c, :, :, :]

            # Computo la operación de convolución para esta ventana
            grad_loss_input[i, c, h, w] = convolve(slice_from_input_padded, filter, 0)
      



      #grad_loss_filter
      aux_loss_out = np.zeros([1, n_C, n_H + self.stride-1, n_W + self.stride-1])

      (1, n_C, n_H_dilate, n_W_dilate) = aux_loss_out.shape


      n_H_prev = int((n_H_padded - filter_size) + 1)
      n_W_prev = int((n_W_padded - filter_size) + 1)
      grad_loss_filter = np.zeros([1, n_C, n_H_prev, n_W_prev])

      # convolve

      # Itero sobre el eje vertical del volumen de salida
      for h in range(n_H_padded):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = h
        y_end = y_start + filter_size

        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W_padded):
          # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          x_start =  w
          x_end = x_start + filter_size

          # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
          slice_from_input_padded = aux_loss_out[i, :, y_start:y_end, x_start:x_end]
          
          # Itero sobre la cantidad de canales del volumen de salida
          for c in range(n_C):

            # Obtengo el valor del filtro y bias del canal correspondiente
            filter = w_flipped[c, :, :, :]

            # Computo la operación de convolución para esta ventana
            grad_loss_filter[i, c, h, w] = convolve(slice_from_input_padded, filter, 0)


      
      return grad_loss_input, grad_loss_filter

  